# <center> Содержание <a id='title'>

+ [Unit 1. Введение](#unit1)  
+ [Unit 2. Сортировка данных в DataFrame](#unit2)  
+ [Unit 3. Группировка данных в DataFrame](#unit3)  
+ [Unit 4. Сводные таблицы](#unit4)  
+ [Unit 5. Обьединение DataFrame: знакомимся с новыми данными](#unit5)  
+ [Unit 6. Обьединение DataFrame: concat](#unit6)  
+ [Unit 7. Обьединение DataFrame: join,merge](#unit7)  
+ [Unit 8. Закрепение знаний](#unit8)  

# <center> [Unit 1. Введение](#title) <a id='unit1'>

Содержание:
+ [Вспомним, с какими данными мы работаем](#unit11)  
+ [Задания](#unit12)

В этом модуле мы продолжим изучение библиотеки Pandas и рассмотрим более сложные методы работы с данными, которые помогают производить их анализ и подготовку. Модуль можно условно разделить на две части:

+ в первой части мы поговорим о продвинутых методах Pandas, позволяющих производить более детальный анализ зависимостей в данных и делать на их основе выводы;
+ вторая часть посвящена работе с несколькими таблицами и их объединению.

Как мы уже говорили, этап подготовки данных играет важную роль в Data Science. Однако этот этап не пройдёт успешно без тщательного анализа зависимостей в данных. Для успешного преобразования таблицы вам необходимо иметь представление о том, как признаки связаны между собой и с целевым признаком, который вы хотели бы предсказать. Поэтому в первой части модуля мы рассмотрим основные инструменты, позволяющие производить такой анализ, а также методы сортировки данных.

Другая проблема заключается в том, что ваши данные не всегда могут быть представлены в виде одной таблицы. Такое нередко случается при работе с выгрузками из целой базы данных. Например, к вам в руки могут попасть несколько таблиц с информацией о клиентах, об их заказах и о поставщиках. Однако вашей целью может быть анализ данных о поставках в целом при использовании всей предоставленной информации. В таком случае таблицы необходимо объединить в единую структуру — об этом мы и поговорим во второй части модуля.

Цели данного модуля:

+ Научиться сортировать данные.
+ Получить навыки группировки данных и анализа групп.
+ Освоить методы построения сводных таблиц и научиться с их помощью исследовать зависимости в признаках.
+ Научиться работать с данными, представленными в виде нескольких таблиц, и познакомиться с методами их объединения.

## <center> [ВСПОМНИМ, С КАКИМИ ДАННЫМИ МЫ РАБОТАЕМ](#unit1) <a id='unit11'>

В этом модуле мы продолжим нашу работу с датасетом о продажах объектов недвижимости в Мельбурне и его пригородах. Информацию об исходных данных вы можете найти [здесь](https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DSPR-2.0+14JULY2021/jump_to_id/e756e694a15846d0bc80a1a0f42c40e6).

В прошлом модуле мы совершили множество преобразований над нашей таблицей — давайте вспомним их:

+ удалили столбцы index и Coordinates;
+ создали признак средней площади одной комнаты MeanRoomsSquare и ввели коэффициент соотношения площади здания к площади участка — AreaRatio;
+ преобразовали признак даты продажи Date в формат datetime и создали на его основе следующие столбцы: номер месяц продажи (MonthSale), номер дня недели продажи (WeekdaySale), «признак-мигалку» выходного дня (Weekend);
+ заменили признак года постройки здания YearBuilt на его возраст AgeBuilding;
+ извлекли из признака адреса объекта Address новый признак подтипа улицы StreetType и удалили столбец с адресом;
+ уменьшили число уникальных наименований агентств по недвижимости (SellerG), а также число пригородов (Suburb);
+ выделили категориальные признаки и преобразовали их в тип данных сategory;
+ заменили сокращённые названия категорий признака типа объекта Type на их полные названия (h — house, t —  townhouse, u — unit).

Скачать преобразованный набор данных в формате csv можно [здесь](https://lms.skillfactory.ru/assets/courseware/v1/87856cc090b7a67a7a0689b0415c8331/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/melb_data_fe.zip) (csv-файл находится в zip-архиве — распакуйте архив, прежде чем продолжать работу!)

Прочитаем датасет и выведем первые пять его строк:

In [ ]:
import pandas as pd
melb_df = pd.read_csv('data\melb_data_fe.csv')
melb_df.head()

Файл подгрузился верно, однако есть один нюанс. Давайте посмотрим на информацию о столбцах с помощью метода info():

In [ ]:
melb_df.info()

Обратите внимание, что наше преобразование столбцов к типам datetime и category «слетело».

    Почему это произошло?

Ответ на самом деле очень прост: csv-файл не хранит в себе информацию о типах данных столбцов, поэтому при чтении Pandas автоматически определяет тип данных столбца. Не забывайте об этом, обмениваясь преобразованными данными с вашими коллегами.

Преобразуем признак Data к типу данных Datatime:

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'])


Преобразум признаки у которых число уникальных категорий меньше 150 к типу данных category: 

In [ ]:
cols_to_exclude = [ 'Rooms','Date', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца

display(melb_df.info()) # Проверим результат

## <center> [ЗАДАНИЯ](#unit1) <a id='unit12'>

### Задание 1.1

Выделите квартал (quarter) продажи объектов недвижимости. Найдите второй по популярности квартал продажи. В качестве ответа запишите число объектов, проданных в этом квартале.

In [ ]:
quarter_sell = melb_df['Date'].dt.quarter
quarter_sell.value_counts()

### Задание 1.2  
Количество столбцов, которые имеют тип данных category:

In [ ]:
melb_df.dtypes.to_list().count('category')

# <center> [Unit 2. Сортировка данных в DataFrame](#title) <a id='unit2'>

Содержание:
+ [Метод Sort_Values()](#unit21)  
+ [Сортировка по значениям одного столбца](#unit22)  
+ [Сортировка по значениям нескольких столбцов](#unit23)  
+ [Комбинирование сортировки с фильтрацией](#unit24)  
+ [Задания](#unit25)

## <center> [МЕТОД SORT_VALUES()](#unit2) <a id='unit21'>

Для сортировки значений в DataFrame по значениям одного или нескольких столбцов используется метод sort_values().

Основные параметры метода sort_values():

- by — имя или список имён столбцов, по значениям которых производится сортировка.
- axis — ось, по которой производится сортировка (0 — строки, 1 — столбцы). По умолчанию сортировка производится по строкам.
- ascending — сортировка по возрастанию (от меньшего к большему). По умолчанию параметр выставлен на True, для сортировки по убыванию (от большего к меньшему) необходимо выставить его на False.
- ignore_index — создаются ли новые индексы в таблице. По умолчанию выставлен на False и сохраняет индексы изначальной таблицы.
- inplace — производится ли замена исходной таблицы на отсортированную. По умолчанию параметр выставлен на False, то есть замены не производится. Чтобы переопределить исходную таблицу на отсортированную, необходимо выставить этот параметр на True.

## <center> [СОРТИРОВКА ПО ЗНАЧЕНИЯМ ОДНОГО СТОЛБЦА](#unit2) <a id='unit22'>

Приведём несколько примеров сортировки нашей таблицы с недвижимостью.

Отсортируем таблицу по возрастанию цены объектов недвижимости (Price):

In [ ]:
melb_df.sort_values(by='Price').head(10)

Мы вывели десять строк таблицы, чтобы убедиться в верном порядке сортировки. Также обратите внимание на индексы таблицы — их значения сохранились из исходной таблицы.  

А теперь отсортируем таблицу по убыванию (от самой последней до самой первой) даты продажи объекта (Date). Для этого выставим параметр ascending на False:

In [ ]:
melb_df.sort_values(by='Date', ascending=False)

## <center> [СОРТИРОВКА ПО ЗНАЧЕНИЯМ НЕСКОЛЬКИХ СТОЛБЦОВ](#unit2) <a id='unit23'>

Для сортировки по значениям нескольких столбцов необходимо передать названия этих столбцов в параметр by в виде списка. При этом важно обращать внимание на порядок следования столбцов.

Так, например, отсортируем таблицу сначала по возрастанию расстояния от центра города (Distance), а затем — по возрастанию цены объекта (Price). Для того чтобы вывод был более наглядным, выделим каждую десятую строку из столбцов Distance и Price результирующей таблицы:

In [ ]:
melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

Мы получили таблицу, отсортированную по возрастанию расстояния до центра города. Если встречаются объекты недвижимости, у которых расстояние оказывается одинаковым, то внутри такой группы производится сортировка по цене объекта

## <center> [КОМБИНИРОВАНИЕ СОРТИРОВКИ С ФИЛЬТРАЦИЕЙ](#unit2) <a id='unit24'>

А теперь рассмотрим применение сортировки на практике.

Предположим, компания McGrath поручила нам восстановить хронологию продаж таунхаусов, у которых площадь участка существенно больше площади здания, чтобы понять, как часто компания справляется с таким сложным видом объектов. Объекты, проданные в один и тот же день, мы бы хотели сортировать по значению коэффициента соотношения площадей. 

Найдём информацию о таунхаусах (Type), проданных компанией (SellerG) McGrath, у которых коэффициент соотношения площадей здания и участка (AreaRatio) меньше -0.8. Результат отсортируем по дате продажи (Date) в порядке возрастания, а после проведём сортировку по убыванию коэффициента соотношения площадей. Также обновим старые индексы на новые, установив параметр ignore_index на True. Для наглядности результата выберем из таблицы только столбцы Data и AreaRatio:

In [ ]:
mask1 = melb_df['AreaRatio'] < -0.8 # создаем фильтр по условию "коэффициент площади меньше 0.8"
mask2 = melb_df['Type'] == 'townhouse' # создаем фильтр по условию "тип недвижимости townhouse"
mask3 = melb_df['SellerG'] == 'McGrath' # создаем фильтр по условию "McGrath -компания по продаже"

# применяем фильтрацию и сортируем по призкам Date, AreaRatio:
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

Примечание: старайтесь не сочетать фильтрацию и метод sort_values() с параметром inplace=True, так как в таком случае у вас возникнет предупреждение (warning) SettingWithCopyWarning: melb_df[melb_df['Rooms'] > 5].sort_values(inplace=True, by=['Rooms']) 
C:\Users\Andrey\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingwithCopyWarning:

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ indexing. html#retu rning-a-view-versus-a-copy

    return func(*args, **kwargs)

Стоит отметить, что это не ошибка и код в таком случае отработает. Однако, Pandas предупреждает вас о том, что при использовании такого кода дальнейшие результаты работы могут быть неожидаемыми.

Если не вдаваться в подробности, то данное предупреждение предназначено для обозначения операций "цепного присваивания". Это ситуация, в которой вы пытаетесь напрямую изменить подмножество исходных данных. В нашем случае мы пытаемся отсортировать с заменой (об этом говорит параметр inplace=True) с заменой исходной таблицы на отсортированную.

Решается данная проблема очень просто: чтобы не возникало подобных конфликтов, мы используем метод copy(), для явного создания копии отфильтрованного подмножества исходных данных и работаем уже с ней (производим сортировку):

filtered = melb_df[melb_df['Rooms'] > 5].copy()

filtered.sort_values(inplace=True, by=['Rooms'])

filtered.head()

Подробнее почитать о предупреждении SettingWithCopyWarning вы можете [здесь](https://newtechaudit.ru/pandas-dataframes/).

## <center> [ЗАДАНИЯ](#unit2) <a id='unit25'>

### Задание 2.2

Произведите сортировку столбца AreaRatio по убыванию. При этом индексы полученной таблицы замените на новые. Какое значение площади здания находится в строке 1558? Ответ округлите до целого числа.

In [ ]:
melb_df.sort_values(by='AreaRatio',ignore_index=True, ascending=False)['BuildingArea'].iloc[1558]

### Задание 2.3

Найдите таунхаусы (Type) с количеством жилых комнат (Rooms) больше 2. Отсортируйте полученную таблицу сначала по возрастанию числа комнат, а затем по убыванию средней площади комнат (MeanRoomsSquare). Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18? Ответ запишите в виде целого числа.

In [ ]:
mask_rooms = melb_df['Rooms'] > 2 # создаем фильтр по условию "количество комнат больше 2"
mask_type = melb_df['Type'] == 'townhouse'  # создаем фильтр по условию "тип недвижимости townhouse"
# применяем фильтрацию и сортируем по призкам Rooms, MeanRoomsSquare:
melb_df[mask_rooms & mask_type].sort_values(
    by=['Rooms','MeanRoomsSquare'],
    ascending= [True, False],
    ignore_index=True
).loc[18,'Price']

# <center> [Unit 3. Группировка данных в DataFrame](#title) <a id='unit3'>

Содержание:
+ [Метод groupby()](#unit31)  
+ [Группировка данных по одному критерию с одной агрегацией](#unit32)  
+ [Группировка данных по одному критерию с несколькми агрегаиями](#unit33)  
+ [Задания](#unit34)

Одна из основных задач анализа данных — это группировка данных и сравнение показателей в группах. Например, нам необходимо сравнить средний уровень заработной платы в зависимости от пола/уровня образования. Или же мы хотим проследить, какая группа клиентов приносит нам наибольший доход, чтобы направить своё внимание на эту группу.

В некоторых случаях группировки может быть достаточно, чтобы ответить на вопросы бизнеса. В других случаях это может стать первым шагом в более сложном анализе. Так, например, благодаря группировке мы можем выявлять признаки, которые не несут статистической значимости, или признаки, которые вносят наибольший вклад. 

Так или иначе, владение группировкой — важный навык, который открывает новые возможности по работе с данными.

## <center> [МЕТОД GROUPBY()](#unit3) <a id='unit31'> 

В библиотеке Pandas для группировки данных по одному или нескольким признакам можно использовать метод [groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html).

Основные параметры метода groupby():

- by — имя или список имён столбцов, по которым производится группировка.
- axis — ось, по которой производится группировка (0 — строки, 1 — столбцы). По умолчанию группировка производится по строкам.
- as_index — добавляется ли дополнительный индекс к таблице. По умолчанию установлен на True.

Метод groupby() возвращает объект DataFrameGroupBy, который хранит в себе информацию о том, какие строки относятся к определённой группе, и сам по себе не представляет для нас интереса. 

Однако к этому объекту можно применять уже знакомые нам агрегирующие методы (mean, median, sum и т. д.), чтобы рассчитывать показатели внутри каждой группы.

Посмотрим на общую схему работы метода groupby() в Pandas:

<center>

![](https://lms.skillfactory.ru/assets/courseware/v1/09fc73c845fb48dc9dc3faab5a960f20/asset-v1%3ASkillFactory%2BDSPR-2.0%2B14JULY2021%2Btype%40asset%2Bblock/dst3-u1-md12_3_1.png)

Сначала мы разделяем данные на группы с помощью метода groupby(), после чего к каждой группе применяем агрегацию и объединяем результаты в новую таблицу.

## <center> [ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С ОДНОЙ АГРЕГАЦИЕЙ](#unit3) <a id='unit32'>


Рассмотрим группировку данных на примере нашей таблицы с недвижимостью.

Применим агрегирующую функцию среднего к результату работы groupby(). В качестве столбца для группировки возьмём столбец типа объекта недвижимости (Type):

In [ ]:
melb_df.groupby(by='Type').mean()

Мы получили таблицу, на пересечении строк и столбцов которой находятся средние значения каждого числового признака в наших данных.

Обратите внимание на структуру получившейся таблицы: теперь на месте индексов стоят значения типа объекта недвижимости Type (house, townhouse, unit).

Примечание. Если мы хотим видеть тип объекта в качестве отдельного столбца таблицы, мы можем выставить параметр as_index на False.

Как правило, нам не нужна информация обо всех столбцах, поэтому агрегирующие методы можно применять только к интересующему нас столбцу. Например, давайте сравним средние цены на объекты в зависимости от их типа:

In [ ]:
melb_df.groupby('Type')['Price'].mean()

Примечание. Обратите внимание, что, так как мы считаем только один показатель (среднее) для одного столбца, в результате мы получаем объект Series.

Теперь давайте выясним, какие регионы (Regionname) наиболее удалены от центра Мельбурна.

Для этого найдём минимальное значение расстояния от центра города до объекта в зависимости от его региона. Результат отсортируем по убыванию расстояния:

In [ ]:
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)
# min() - сначала находим ближающую к центру "точку" для каждого региона
# sort_values - сортируем по убыванию, находя самый удаленный район

Итак, наиболее удалёнными являются все регионы Victoria.

## <center> [ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ](#unit3) <a id='unit33'>

Чтобы рассчитать несколько агрегирующих методов, можно воспользоваться методом [agg()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html), который принимает список строк с названиями агрегаций.

Давайте построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж, а также среднее и максимальное значения цен объектов недвижимости (Price), сгруппированных по номеру месяца продажи (MonthSale). Результат отсортируем по количеству продаж в порядке убывания:

In [ ]:
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

Примечание. Обратите внимание, что, так как мы считаем несколько показателей для одного столбца, в результате мы получаем объект DataFrame.

В результате применения метода agg(), в который мы передали список с названиями интересующих нас агрегирующих функций, мы получаем DataFrame со столбцами count, mean и max, где для каждого месяца рассчитаны соответствующие параметры. Результат сортируем по столбцу count.

    Какие интересные выводы можно сделать из этой таблицы?

1. Пик продаж приходится на период весна-лето.

2. Средняя цена продаваемых объектов относительно стабильна и находится в пределах 1 млн. австралийских долларов с небольшими отклонениями (около 100 тыс. влево и вправо).

3. Прослеживается некоторая зависимость между сезоном и максимальной ценой объектов: в месяцы с большим спросом на объекты недвижимости цена также имеет наибольшие показатели. Можно сделать предположение, что это связано с повышением цен на элитные дома в периоды большого спроса.

Если нужна полная информация обо всех основных статистических характеристиках внутри каждой группы, вы можете воспользоваться методом agg(), передав в качестве его параметра строку 'describe':

In [ ]:
melb_df.groupby('MonthSale')['Price'].agg('describe')

После базовых математических функций наиболее частым агрегированием является подсчёт числа уникальных значений. Так, например, мы можем вычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять, в каких регионах конкуренция на рынке недвижимости меньше. Это можно сделать, передав в параметр метода agg() строку 'nunique'. 

Более того, метод agg() поддерживает использование и других функций. Передадим дополнительно встроенную функцию set, чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов:

In [ ]:
melb_df.groupby('Regionname')['SellerG'].agg(['nunique', set])

Как и ожидалось, наименьшая конкуренция в наиболее удалённом регионе Western Victoria, а наибольшая — в центральном районе Northern Metropolitan.

## <center> [ЗАДАНИЯ](#unit3) <a id='unit34'>

### Задание 3.1

Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе. В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.

In [ ]:
melb_df.groupby('Rooms').mean().sort_values('Price',ascending=False)
#.groupby('Price').max()

### Задание 3.2

Какой регион имеет наименьшее стандартное отклонение по географической широте (Lattitude)?
В качестве ответа запишите название этого региона.

In [ ]:
melb_df.groupby('Regionname').std().sort_values('Lattitude')

### Задание 3.3

Какая риелторская компания (SellerG) имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?

Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании в заданный период.
Не забудьте перевести даты в формат datetime.

In [ ]:
mark_date1 = melb_df['Date'] >= '2017-05-01' # фильтр по условию
mark_date2 = melb_df['Date'] <= '2017-09-01' # фильтр по условию
melb_df[mark_date1 & mark_date2].groupby('SellerG').sum()["Price"].sort_values()

# <center> [Unit 4. Сводные таблицы](#title) <a id='unit4'>

Содержание:
+ [Метод groupby() как способ построения сводных таблиц](#unit41)  
+ [Метод pivot_table() для построения сводных таблиц](#unit42)  
+ [Многомерные сводные таблицы](#unit43)  
+ [Доступ к данным в сводной таблице](#unit44)  
+ [Задания](#unit45)

Сводные таблицы — это распространённый инструмент для агрегации данных.

Сводная таблица принимает на вход данные из отдельных столбцов и группирует их. В результате получается новая таблица, которая позволяет увидеть многомерное обобщение данных. Таким образом, благодаря сводным таблицам мы можем оценить зависимость между двумя и более признаками данных.

Мы чаще сталкиваемся со сводными таблицами, чем с обычными, в плоском виде, так как сводные таблицы удобнее для анализа и быстрых выводов, а также позволяют увидеть более общие зависимости между признаками, нежели простая группировка данных.

Инструмент сводных таблиц также широко популярен среди тех, кто использует Excel или какие-либо BI-системы.

## <center> [МЕТОД GROUPBY КАК СПОСОБ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ](#unit4) <a id='unit41'> 

На самом деле мы с вами уже строили простейшие одномерные сводные таблицы с помощью метода groupby — мы рассматривали сводную таблицу в контексте группировки по одному признаку. 

Например, мы уже умеем строить таблицу, которая показывает зависимость медианной цены и площади здания от числа комнат:

In [ ]:
melb_df.groupby('Rooms')[['Price', 'BuildingArea']].median()

Также можно построить таблицу, в которой мы будем учитывать не только число комнат, но и тип здания (Type). Для этого в параметрах метода groupby() укажем список из нескольких интересующих нас столбцов.

In [ ]:
melb_df.groupby(['Rooms', 'Type'])['Price'].mean()

В результате выполнения такого кода мы получаем Series, которая обладает несколькими уровнями индексов: первый уровень — число комнат, второй уровень — тип здания. Такая организация индексов называется иерархической. Вычисление параметра (средней цены) происходит во всех возможных комбинациях признаков.

Для того, чтобы финальный результат был представлен в виде сводной таблицы (первый группировочный признак по строкам, а второй — по столбцам), а не в виде Series с иерархическими индексами, к результату чаще всего применяют метод [unstack()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.unstack.html), который позволяет переопределить вложенный индекс в виде столбцов таблицы:

In [ ]:
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

В результате мы получаем сводную таблицу, столбцы в которой представляют типы домов (house, townhouse, unit), строки — число комнат, а на пересечении строк и столбцов находится средняя стоимость объекта с такими показателями.

Какие интересные выводы можно сделать из этой таблицы?

- Пропуски в сводной таблице (NaN) говорят о том, что в наших данных нет соответствующих комбинаций признаков. Например, у нас нет информации о ценах на таунхаусы, где количество комнат больше пяти.

- Наибольшей средней стоимостью (2,25 млн. австралийских долларов) обладают объекты типа unit с восемью жилыми комнатами. Наименьшая средняя стоимость — у однокомнатных домов типа unit (чуть меньше 400 тыс. австралийских долларов).

- Сколько бы комнат ни было в доме, цена на объекты типа unit всегда ниже других (за исключением восьмикомнатных объектов).

## <center> [МЕТОД PIVOT_TABLE ДЛЯ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ](#unit4) <a id='unit42'>

На самом деле метод groupby редко используется при двух параметрах, так как для построения сводных таблиц существует специальный и более простой метод — [pivot_table()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html).

Основные параметры метода pivot_table():
- values — имя столбца, по которому необходимо получить сводные данные, применяя агрегирующую функцию;
- index — имя столбца, значения которого станут строками сводной таблицы;
- columns — имя столбца, значения которого станут столбцами сводной таблицы;
- aggfunc — имя или список имён агрегирующих функций (по умолчанию — подсчёт среднего, 'mean');
- fill_value — значение, которым необходимо заполнить пропуски (по умолчанию пропуски не заполняются).

Давайте построим ту же самую таблицу, но уже с использованием метода pivot_table. В качестве параметра values укажем столбец Price, в качестве индексов сводной таблицы возьмём Rooms, а в качестве столбцов — Type. Агрегирующую функцию оставим по умолчанию (среднее). Дополнительно заменим пропуски в таблице на значение 0. Финальный результат для наглядности вывода округлим с помощью метода [round()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.round.html) до целых.

In [ ]:
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

Несложно понять, что метод pivot_table() имеет преимущество перед группировкой по нескольким критериям. Оно заключается в наличии специальных аргументов для строк и столбцов сводной таблицы, благодаря чему уменьшается вероятность запутаться при построении более сложных (многомерных) сводных таблиц, о которых мы поговорим далее.

А теперь давайте проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной. Для этого построим сводную таблицу, в которой строками будут являться названия регионов (Regionname), а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend), который равен 1, если день был выходным, и 0 — в противном случае. В качестве значений сводной таблицы возьмём количество продаж.

In [ ]:
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count',
)

Из результирующей таблицы можно сделать два вывода:

- Число продаж резко возрастает в выходные вне зависимости от региона (приблизительно в 2-3 раза). То есть вероятность того, что дом продадут в выходные, гораздо выше вероятности, что его продадут в будний день.

- В отдалённых регионах (Victoria) коэффициент роста числа продаж выше, чем в центральных. Если в центральных регионах Metropolitan продажи по выходным в 2-2.5 раза выше, чем по будням, то в регионах Victoria число продаж в выходные вырастает примерно в 3 раза.

Такой рост можно даже попытаться объяснить логически: в выходные дни у людей появляется свободное время, чтобы доехать до отдалённых пригородов с целью покупки дома.

Разберём ещё один пример: найдём, как зависит средняя и медианная площадь участка (Landsize) от типа объекта (Type) и его региона (Regionname). Чтобы посмотреть несколько статистических параметров, нужно передать в аргумент aggfunc список из агрегирующих функций. Построим такую сводную таблицу, где пропущенные значения заменим на 0:

In [ ]:
melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

Обратите внимание на добавление дополнительных индексов столбцов median и mean. Здесь медианное и среднее значения рассчитаны отдельно для каждой комбинации признаков.

Здесь в глаза бросаются объекты типа house в регионах Eastern Victoria и Northern Victoria — в них среднее и медиана отличаются более чем в три раза. Вероятно, это связано с тем, что в этих районах очень большой разброс цен: есть несколько объектов с гигантской площадью, а остальные объекты имеют небольшую площадь. Из-за этого среднее значение искажается, в то время как медиана нечувствительна к такому разбросу и не искажает результат.

## <center> [МНОГОМЕРНЫЕ СВОДНЫЕ ТАБЛИЦЫ](#unit4) <a id='unit43'>

До этого мы рассматривали, как некоторый статистический показатель может зависеть от двух признаков. Однако, как уже упоминалось, сводные таблицы позволяют наблюдать зависимость и от большего числа признаков. Такие сводные таблицы называются многомерными. 

Для того чтобы исследовать зависимость от большего числа признаков, можно передать список признаков в параметр index или параметр columns.

Давайте построим таблицу, в которой по индексам будут располагаться признаки метода продажи (Method) и типа объекта (Type), по столбцам — наименование региона (Regionname), а на пересечении строк и столбцов будет стоять медианная цена объекта (Price):

In [ ]:
melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

Первым индексом в таблице идёт метод продажи здания, далее для метода указывается тип недвижимости. По столбцам расположены регионы. В ячейках таблицы указана медианная цена для каждой такой комбинации.

Такие таблицы уже сложнее читать, однако с помощью них можно более глубоко исследовать закономерности. Например, можно видеть, что вне зависимости от метода продажи и региона цена на объекты типа house практически всегда выше, чем на объекты другого типа.

## <center> [ДОСТУП К ДАННЫМ В СВОДНОЙ ТАБЛИЦЕ](#unit4) <a id='unit44'>

    Как получить доступ к данным или произвести фильтрацию в сложной сводной таблице, где есть дополнительные индексы?
    
Давайте рассмотрим, что собой представляют столбцы сложной сводной таблицы.

Запишем сводную таблицу, которую мы создавали ранее в переменную pivot:

In [ ]:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

Выведем её столбцы с помощью атрибута columns:

In [ ]:
pivot.columns

В результате мы получаем объект MultiIndex. Этот объект хранит в себе шесть комбинаций пар столбцов (два статистических параметра и три типа здания), то есть есть шесть возможных вариантов обращения к столбцам таблицы.

Мультииндексы раскрываются подобно вложенным словарям — по очереди, как матрёшка. Чтобы получить доступ к определённому столбцу, вы должны сначала обратиться к столбцу, который находится уровнем выше.

Так, из таблицы pivot мы можем получить средние значения площадей участков для типа здания unit, просто последовательно обратившись по имени столбцов:

In [ ]:
display(pivot['mean']['unit'])

Аналогично производится и фильтрация данных. Например, если нам нужны регионы, в которых средняя площадь здания для домов типа house меньше их медианной площади, то мы можем найти их следующим образом:

In [ ]:
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

Чтобы получить индексы отфильтрованной таблицы, можно воспользоваться атрибутом index и обернуть результат в список:

In [ ]:
print(list(filtered_pivot.index))

Таким образом, сводные таблицы изначально кажутся сложной структурой, но на самом деле это обычные DataFrame со вложенными индексами строк или столбцов. 

Умение читать и анализировать сложные сводные таблицы — это важный навык, который помогает проводить углублённый анализ данных.

На самом деле мультииндексные таблицы можно создавать и вручную. Давайте посмотрим на синтаксис данной конструкции:

In [ ]:
import numpy as np
mser = pd.Series(
    np.random.rand(8),
	index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']])
display(mser)

В данном примере мы создаём объект Series со вложенными индексами. Мы передаём в качестве индексов Series вложенный список, где первый список задаёт внешний уровень вложенности, а второй список — внутренний уровень вложенности. Значения Series — случайные числа от 0 до 1, сгенерированные функцией np.random.rand() (ваши значения могут отличаться).

Если посмотреть на индексы Series, можно увидеть, что они являются мультииндексами:

In [ ]:
print(mser.index)

Аналогично создаются DataFrame со вложенными признаками (вложенными столбцами) — для этого вложенный список передаётся в параметр columns при инициализации таблицы:

In [ ]:
mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
display(mframe)

## <center> [ЗАДАНИЯ](#unit4) <a id='unit45'>

### Задание 4.2

Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms).
Для какой комбинации признаков площадь здания наибольшая?

In [ ]:
from statistics import median


melb_df.pivot_table(
    values='BuildingArea',
    index='Rooms',
    columns='Type',
    aggfunc=median,
    fill_value=0
)

### Задание 4.3

Составьте сводную таблицу, которая показывает зависимость медианной цены объекта недвижимости (Price) от риелторского агентства (SellerG) и типа здания (Type).
Во вновь созданной таблице найдите агентство, у которого медианная цена для зданий типа unit максимальна. В качестве ответа запишите название этого агентства.

In [ ]:
melb_df.pivot_table(
    values="Price",
    index='SellerG',
    columns='Type',
    aggfunc=median,
    fill_value=0
).sort_values('unit',ascending=False)

# <center> [Unit 5. Объединение DataFrame: знакомимся с новыми данными](#title) <a id='unit5'>

Содержание:
+ [С какими данными мы работаем?](#unit51)  
+ [Зачем хранить данные в разных таблицах](#unit52)  
+ [Задания](#unit53)

На практике источники данных редко ограничиваются одной таблицей. Например, если мы работаем с базой данных, то данные в ней могут быть представлены в виде нескольких десятков таблиц, каждая из которых несёт отдельную информацию. Если вы делаете выгрузку из базы напрямую, не объединяя таблицы в единую структуру средствами SQL, вам необходимо знать, как работать с такими таблицами средствами Pandas. 

## <center>[С КАКИМИ ДАННЫМИ МЫ РАБОТАЕМ?](#unit5) <a id='unit51'>

В этой части модуля мы будем работать с популярным датасетом [MovieLens](https://grouplens.org/datasets/movielens/), в котором собраны логи некоторой рекомендательной системы фильмов.

Скачать таблицы, с которыми мы будем работать, можно [здесь](https://lms.skillfactory.ru/assets/courseware/v1/bcd86def807792174f50d955515c8be0/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/movies_data.zip) (файлы находятся в zip-архиве — распакуйте архив, прежде чем продолжать работу!)

Наши данные представляют собой четыре таблицы:

1. ratings1 и ratings2 — таблицы с данными о выставленных пользователями оценках фильмов.
Они имеют одинаковую структуру и типы данных — на самом деле это две части одной таблицы с оценками фильмов:

- userId — уникальный идентификатор пользователя, который выставил оценку;
- movieId — уникальный идентификатор фильма;
- rating — рейтинг фильма.

2. dates — таблица с датами выставления всех оценок:
- date — дата и время выставления оценки фильму.

3. movies — таблица с информацией о фильмах:

- movieId — уникальный идентификатор фильма;
- title — название фильма и год его выхода;
- genres — жанры фильма.

Итак, представим, что нам надо получить единую таблицу, в которой будут собраны рейтинги, даты выставления рейтингов, а также информация о фильмах. 
Вот как мы будем действовать:
1. Склеим таблицы ratings1 и ratings2 в единую структуру.
2. К полученной таблице с рейтингами подсоединим столбец с датой проставления рейтинга, склеив столбцы таблиц между собой.
3. Присоединим к нашей таблице информацию о названиях и жанрах фильмов.

## <center>[ЗАЧЕМ ХРАНИТЬ ДАННЫЕ В РАЗНЫХ ТАБЛИЦАХ?](#unit5) <a id='unit52'>

Конечно, здорово, если все необходимые данные лежат в одной таблице, но на практике такое случается редко по двум объективным причинам:

1. Часто данные формируются несколькими независимыми процессами, каждый из которых хранит данные в своей таблице.

Например, данные для отчёта по продажам могут состоять из списка банковских транзакций, курсов валют от Центробанка и планов отдела продаж из внутренней CRM. Все эти три таблицы, скорее всего, будут формироваться независимыми друг от друга системами. Объединять их в один отчёт придётся вам.

2. Хранить все данные в одной таблице часто очень накладно для ёмкости диска.

Например, названия фильмов в наших данных хранятся в отдельной небольшой таблице. А в логах, которые могут растягиваться на многие миллионы строк, вместо названия фильма стоит его идентификатор. Числовой идентификатор фильма занимает на диске гораздо меньше места, чем длинное название, поэтому логи с идентификаторами будут занимать гораздо меньше места, чем единая таблица с названиями.

## <center>[ЗАДАНИЯ](#unit5) <a id='unit53'>

### Задание 5.2

Сколько уникальных фильмов представлено в таблице movies?

In [ ]:
import pandas as pd
movies = pd.read_csv('data\movies_data\movies.csv')
#movies['title'].value_counts()
movies[movies['title']=='Emma (1996)'].index.to_list()


### Задание 5.3

Сколько уникальных пользователей в таблице ratings1?

In [ ]:
ratings1 = pd.read_csv('data\movies_data\\ratings1.csv')
ratings1['userId'].nunique()


### Задание 5.4

В каком году было выставлено больше всего оценок?
Для ответа на этот вопрос используйте таблицу dates.

In [ ]:
datess = pd.read_csv('data\movies_data\dates.csv')
datess['date'] = pd.to_datetime(datess['date'])
datess['Year'] = datess['date'].dt.year
datess['Year'].value_counts()

# <center> [Unit 6. Обьединение DataFrame: concat](#title) <a id='unit6'>

Следуя нашему плану объединения таблиц, первым делом мы должны склеить таблицы ratings1 и ratings2 по строкам.

Для этого воспользуемся встроенной функцией Pandas concat(), которая позволяет склеивать (конкатенировать) таблицы как по строкам, так и по столбцам.

Основные параметры функции concat():

- objs — список объектов DataFrame ([df1, df2,…]), которые должны быть сконкатенированы;
- axis — ось определяет направление конкатенации: 0 — конкатенация по строкам (по умолчанию), 1 — конкатенация по столбцам;
- join — либо inner (пересечение), либо outer (объединение); рассмотрим этот момент немного позже;
- ignore_index — по умолчанию установлено значение False, которое позволяет значениям индекса оставаться такими, какими они были в исходных данных. Если установлено значение True, параметр будет игнорировать исходные значения и повторно назначать значения индекса в последовательном порядке.

Для корректной конкатенации по строкам объединяемые таблицы должны иметь одинаковую структуру — идентичное число и имена столбцов.

Итак, давайте склеим  ratings1 и ratings2 по строкам, так как они имеют одинаковую структуру столбцов. Для этого передадим их списком в функцию concat(). Помним, что параметр axis по умолчанию равен 0, объединение происходит по строкам, поэтому не трогаем его. 

Примечание. Обратите внимание, что concat является функцией библиотеки, а не методом DataFrame. Поэтому её вызов осуществляется как pd.concat(...).

In [ ]:
import pandas as pd
dates = pd.read_csv('data\movies_data\dates.csv')
movies = pd.read_csv('data\movies_data\movies.csv')
ratings1 = pd.read_csv('data\movies_data\\ratings1.csv')
ratings2 = pd.read_csv('data\movies_data\\ratings2.csv')
ratings = pd.concat([ratings1, ratings2])
display(ratings)

В результате мы увеличили первую таблицу, добавив снизу строки второй таблицы.

На первый взгляд может показаться, что всё прошло успешно, однако если мы посмотрим на индексы последних строк таблицы, то увидим, что их нумерация не совпадает с количеством строк. Это может привести к некорректному объединению таблиц по ключевым столбцам на следующем этапе решения нашей задачи.

Это связано с тем, что по умолчанию concat сохраняет первоначальные индексы объединяемых таблиц, а обе наши таблицы индексировались, начиная от 0. Чтобы создать новые индексы, нужно выставить параметр ignore_index на True:

In [ ]:
ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
display(ratings)

Казалось бы, совсем другое дело! Но это ещё не всё. Давайте узнаем количество строк в таблицах ratings и dates, ведь нам предстоит вертикально склеить их между собой:

In [ ]:
print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates.shape[0])
print(ratings.shape[0] == dates.shape[0])

Размерность таблиц разная — как такое могло произойти?

На самом деле очень просто: при выгрузке данных информация об оценках какого-то  пользователя попала в обе таблицы (ratings1 и ratings2). В результате конкатенации случилось дублирование строк. В данном примере их легко найти — выведем последнюю строку таблицы ratings1 и первую строку таблицы ratings2:

In [ ]:
display(ratings1.tail(1))
display(ratings2.head(1))

Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame [drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html), который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей, выставив параметр ignore_index в методе drop_duplicates() на значение True:

In [ ]:
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

Наконец, мы можем добавить к нашей таблице с оценками даты их выставления. Для этого конкатенируем таблицы ratings и dates по столбцам:

In [ ]:
ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(7))

 Итак, мы смогли создать единую таблицу с рейтингами и датами их представления. Нашим следующим шагом будет присоединить к таблице информацию о фильмах из таблицы movies.

## <center> [ЗАДАНИЯ](#unit6)

### ЗАДАНИЕ 6.3

Допустим, в ваше распоряжение предоставлена директория "./Root/users/". В данной директории содержатся csv-файлы, в каждом из которых хранится информация об идентификаторах пользователей (user_id) и ссылки на их фотографии (photo_url). Каждый файл из папки users имеет примерно следующую структуру:

| |user_id|image_url|
|-|:------|--------:|
|0|ID01   |http://example.com/img/id001.png
|1|ID02   |http://example.com/img/id002.png
|2|ID03   |http://example.com/img/id003.png
|3|ID04   |http://example.com/img/id004.png


При проверке в директории может быть сколько угодно файлов (директория может изменяться в зависимости от устройства файловой системы).

Вам необходимо написать функцию concat_user_files(path), параметром которой является path — путь до директории. Функция должна объединить информацию из предоставленных вам файлов в один DataFrame и вернуть его. 

Список названий всех файлов, находящихся в директории, вы можете получить с помощью функции [os.listdir(path)](https://docs-python.ru/standart-library/modul-os-python/funktsija-listdir-modulja-os/) из модуля os (модуль уже импортирован в файле main.py). Например, для директории "./Root/users/" результатом работы функции будет список:

print(os.listdir('./Root/users/'))

Примечание. Модуль os позволяет работать с операционной системой компьютера прямо из кода. Подробнее о нем вы можете почитать [здесь](https://pythonworld.ru/moduli/modul-os.html).

Отсортируйте этот список, прежде чем производить объединение файлов.

Когда вы получите отсортированный список, вам останется только прочитать все csv-файлы из списка в цикле и объединить прочитанные таблицы между собой.

Однако обратите внимание, что метод os.listdir() возвращает только список имён файлов в указанной директории, а при чтении файла необходимо указывать полный путь до него. То есть путь для чтения будет таким:

'./Root/users/{file_name}'

Не забудьте обновить индексы результирующей таблицы после объединения.

Учтите, что на тестовом наборе файлов в результате объединения могут возникнуть дубликаты, от которых необходимо будет избавиться.

In [ ]:
import pandas as pd
import os

def concat_users_files(path):
    files_list = os.listdir(path)
    files_list.sort()
    dates_list = []
    for f_value in files_list:
        dates_list.append(pd.read_csv('{}{}'.format(path,f_value)))
    user_files = pd.concat(dates_list)
    user_files = user_files.drop_duplicates(ignore_index=True)
    return user_files
data = concat_users_files('data/users/')
data

# <center> [Unit 7. Обьединение DataFrame: join,merge](#title) <a id='unit7'>

Содержание:
+ [Типы обьединений](#unit71)  
+ [Метод обьединения join()](#unit72)  
+ [Метод обьединения merge()](#unit73)  
+ [Особенности использования merge()](#unit74)  
+ [Какой метод обьединения использовать?](#unit75)  
+ [Задания](#unit76) 

У таблиц ratings и movies есть общий столбец movieId, который каждому фильму из таблицы movies ставит в соответствие поставленные ему оценки из таблицы ratings. Мы хотим объединить их в единую структуру согласно этому соответствию. Объединения такого рода часто называют объединением по ключевому столбцу.

Однако прежде чем мы перейдём к дальнейшей работе с нашими таблицами о фильмах, мы должны рассмотреть основные типы объединения таблиц.

## <center> [ТИПЫ ОБЪЕДИНЕНИЙ](#unit7) <a id='unit71'>

Типы объединений в Pandas тесно связаны с операцией join из SQL, которую мы будем рассматривать в курсе в дальнейшем.

Они представлены на схеме ниже в виде кругов Эйлера. 

Круги Эйлера — это геометрический способ отобразить отношения между множествами. Мы уже сталкивались с базовыми операциями между множествами (пересечением, объединением и вычитанием), когда говорили о множествах в модуле [PYTHON-2](Abstract_PY_2)\Unit 4. Типы данных: множества

<center>

![](data/dst3-u1-md12_7_1.png)

Прежде чем мы перейдём к дальнейшей работе с таблицами о фильмах, рассмотрим два основных типа объединения таблиц:
- inner (внутреннее):
При использовании такого типа объединения в результирующей таблице остаются только те записи, которые есть в обеих таблицах. Аналогия в теории множеств - пересечение (intersection) множеств А и В. Строки, для которых совпадение не было найдено, удаляются.
- outer (внешнее). Данный тип делится на три подтипа:
    -  full — используется как outer по умолчанию, объединяет все варианты в обеих таблицах. Аналогия в теории множеств - объединение (union) множеств А и В;
    - left — для всех записей из «левой» таблицы (например, ratings) ведётся поиск соответствий в «правой» (например, movies). В результирующей таблице останутся только те значения, которым были найдены соответствия, то есть только значения из ratings. Аналогия в теории множеств - вычитание (difference) множества B из результата объединения (union) множеств А и В;
    - right — аналогично предыдущему, но остаются значения только из «правой» таблицы. Аналогия в теории множеств - вычитание (difference) множества А из результата объединения (union) множеств А и В.

    Во всех трёх случаях, если совпадений между таблицами не найдено, на этом месте ставится пропуск (NaN).

## <center> [МЕТОД ОБЪЕДИНЕНИЯ JOIN](#unit7) <a id='unit72'>

Для объединения двух таблиц по индексам используется метод DataFrame join(). Однако данный метод можно применить и для того, чтобы объединить таблицы по ключевому столбцу (в нашем случае это movieId).

Основные параметры метода join():

- other — таблица, которую мы присоединяем. При объединении она является «правой», а исходная таблица, от имени которой вызывается метод, является «левой»;
- how — параметр типа объединения. Он может принимать значения 'inner', 'left' (left outer), 'right' (right outer), и 'outer' (full outer). По умолчанию параметр установлен на 'left';
- on — параметр, который определяет, по какому столбцу в «левой» таблице происходит объединение по индексам из «правой»;
- lsuffix и rsuffix — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.

Ниже представлена общая схема работы метода join() в зависимости от типа объединения:

![](data/dst3-u1-md12_7_2.png)

Если использовать метод join() «в лоб» (без указания ключевого столбца), то объединение произойдёт, как и задумано — по индексам двух таблиц согласно установленному типу объединения.

Проверим это, объединив таблицы типом left. Так как в наших таблицах есть одноимённые столбцы, установим один из суффиксов, чтобы избежать ошибки:

In [ ]:
joined_false = ratings_dates.join(
    movies,
    rsuffix='_right',
    how='left'
)
display(joined_false)

При объединении таблиц по индексам в результирующую таблицу попали все строки из «левой» таблицы, а недостающие строки из «правой» были заполнены пропусками. Так работает тип объединения left.

Попробуйте изменить тип объединения, чтобы посмотреть на разницу результирующих таблиц.

Обратите внимание, что в данном случае у нас получилось два столбца, соответствующих идентификатору фильма: один — из «левой» таблицы (movieId), а другой — из «правой» (movieId_right).

Однако это не тот результат, который мы хотели, ведь мы не получили соответствия фильмов и их рейтингов. Чтобы совместить таблицы по ключевому столбцу с помощью метода join(), необходимо использовать ключевой столбец в «правой» таблице в качестве индекса. Это можно сделать с помощью метода set_index(). Также необходимо указать название ключа в параметре on.

In [ ]:
joined = ratings_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined.head())

В результате такого объединения для каждого идентификатора фильма movieId в таблице ratings_dates найден совпадающий с ним идентификатор movieId в таблице movies и присоединена информация о самом фильме (title и genres). Это как раз то, что нам нужно.

Обратите внимание, что в результате такого объединения остался лишь один столбец movieId.

Примечание. Join() также можно использовать с параметром how='outer'.

## <center> [МЕТОД ОБЪЕДИНЕНИЯ MERGE](#unit7) <a id='unit73'>

Аналогично предыдущему, метод merge() предназначен для слияния двух таблиц по ключевым столбцам или по индексам. Однако, в отличие от join(), метод merge() предлагает более гибкий способ управления объединением, благодаря чему является более популярным.

Основные параметры метода merge():

- right — присоединяемая таблица. По умолчанию она является «правой»;
- how — параметр типа объединения. По умолчанию принимает значение 'inner';
- on — параметр, который определяет, по какому столбцу происходит объединение. Определяется автоматически, но рекомендуется указывать вручную;
- left_on — если названия столбцов в «левой» и «правой» таблицах не совпадают, то данный параметр отвечает за наименования ключевого столбца исходной таблицы;
- right_on — аналогично предыдущему, параметр отвечает за наименование ключевого столбца присоединяемой таблицы.

Метод merge() в первую очередь предназначен для слияния таблиц по заданным ключам, поэтому он не требует установки ключевых столбцов в качестве индекса присоединяемой таблицы. Кроме того, данный метод позволяет объединять даже таблицы с разноимёнными ключами. Таким образом, merge() проще в использовании и более многофункционален, чем схожие методы.

Посмотрим на метод merge() в действии. Произведём слияние наших таблиц и получим ту же таблицу, что и ранее:

In [ ]:
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
display(merged.head())

Проверим, что число строк в таблице ratings_dates совпадает с числом строк в результирующей таблице merged:

In [ ]:
print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])


Всё прошло успешно: для каждой оценки пользователя мы нашли информацию о фильме, которому она была выставлена.

## <center> [ОСОБЕННОСТИ ИСПОЛЬЗОВАНИЯ MERGE()](#unit7) <a id='unit74'>

Возникает вопрос: почему мы выбрали тип объединения left, а не full, например?

Найти ответ нам поможет пример. Объединим ratings_dates с movies по ключевому столбцу movieId, но с параметром how='outer' (full outer) и выведем размер таблицы, а также её «хвост»:

In [ ]:
merged2 = ratings_dates.merge(
    movies,
    on='movieId',
    how='outer'
)
print('Число строк в таблице merged2: ', merged2.shape[0])
display(merged2.tail())

Результирующее число строк в таблице увеличилось. Но за счёт чего?

Оказывается, в таблице movies содержались фильмы, которым ещё не были выставлены оценки. В результате объединения типом full outer информация о фильмах перенеслась из таблицы movies в результирующую таблицу. Однако, поскольку оценки фильмам ещё не были выставлены, соответствующие столбцы таблицы ratings_dates заполнились пропусками (NaN). Такие фильмы были записаны в конец таблицы.

Важно! Учитывайте такие нюансы при работе с несколькими таблицами и всегда проверяйте результат объединения.

Метод merge() с внешним (outer) типом объединения может использоваться как аналог метода concat() при объединении таблиц с одинаковой структурой (одинаковые количество и названия столбцов) по строкам. В таком случае все одноимённые столбцы таблиц будут считаться ключевыми.

Рассмотрим пример: объединим таблицы ratings1 и ratings2, как мы уже делали раньше, но теперь используем метод merge():

In [ ]:
merge_ratings = ratings1.merge(ratings2, how='outer')
print('Число строк в таблице merge_ratings: ', merge_ratings.shape[0])
display(merge_ratings)

Обратите внимание, что при использовании метода merge() для склейки двух таблиц у нас автоматически пропали дубликаты, которые мы видели при использовании метода concat(). Это особенность метода merge() — автоматическое удаление дублей.

## <center> [КАКОЙ МЕТОД ОБЪЕДИНЕНИЯ ИСПОЛЬЗОВАТЬ?](#unit7) <a id='unit75'>

Итак, мы рассмотрели три основных метода объединения таблиц: concat(), join() и merge(). Давайте структурируем материал, изложенный ранее, в виде небольшой блок-схемы, которая поможет вам определить, какой метод является предпочтительным при объединении таблиц.

![](data/dst3-u1-md12_7_10.png)

## <center> [ЗАДАНИЯ](#unit7) <a id='unit76'>

### Задание 7.5

Даны две таблицы: items_df, в которой содержится информация о наличии товаров на складе, и purchase_df — с данными о покупках товаров.

In [ ]:
items_df = pd.DataFrame({
'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})

In [ ]:
purchase_df = pd.DataFrame({
'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})


Информация в таблицах представлена в виде следующих столбцов:

- item_id — идентификатор модели;
- vendor — производитель модели;
- stock_count — имеющееся на складе количество данных моделей (в штуках);
- purchase_id — идентификатор покупки;
- price — стоимость модели в покупке.

Сформируйте DataFrame merged, в котором в результате объединения purchase_df и items_df останутся модели, которые учтены на складе и имели продажи.

In [ ]:
merged_frame = items_df.merge(
    purchase_df,
    how='inner',
    on='item_id'
)
merged_frame

Из таблицы merged найдите суммарную выручку, которую можно было бы получить от продажи всех товаров, которые учтены на складе и имели продажи. Результат занесите в переменную income.

In [ ]:
merged_frame['total']=merged_frame['stock_count']*merged_frame['price']
income = merged_frame['total'].sum()
print(income)
merged_frame

# <center> [Unit 8. Закрепление знаний](#title) <a id='unit8'>

Результат нашего объединения таблиц с рейтингами вы можете найти [здесь](https://lms.skillfactory.ru/assets/courseware/v1/8e23b4ad7fb8d49f0b9c67be84dc440c/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/ratings_movies.zip) (csv-файл находится в zip-архиве — распакуйте архив, прежде чем продолжать работу!)

In [ ]:
import pandas as pd
rating_movies = pd.read_csv('data\\ratings_movies.csv')
rating_movies.head()

Для решения задач нам понадобится выделить из признака title год выпуска фильма. Для этого напишем функцию get_year_release(arg):

In [ ]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

Данная функция основана на регулярных выражениях и использует модуль re, с которым мы пока не работали.

Модуль re предназначен для поиска шаблонов в тексте и встроен в язык, поэтому не нуждается в установке.

ДОПОЛНИТЕЛЬНО

При желании вы можете ознакомиться с регулярными выражениями по [ссылке](https://tproger.ru/translations/regular-expression-python/), однако это не является обязательным для выполнения заданий в этом юните.

Из модуля re нам понадобится только функция [findall()](https://docs-python.ru/standart-library/modul-re-python/funktsija-findall-modulja-re/), которая позволяет найти в строке все слова, удовлетворяющие шаблону. Мы находим в строке с названием фильма шаблон "(DDDD)" — четыре цифры, обёрнутых в скобки, что соответствует году выпуска фильма. Если такого шаблона не было найдено (год выпуска не указан), функция возвращает None (в таблице это будет помечено как пропуск).

## <center> [ЗАДАНИЯ](#unit8)

### Задание 8.1

Создайте в таблице новый признак year_release, который соответствует году выпуска фильма:

In [ ]:
rating_movies['year_release'] = rating_movies['title'].apply(get_year_release)
rating_movies.tail()

У скольких фильмов не указан год их выпуска?

In [ ]:
rating_movies.info()
100836-100818


### Задание 8.2

Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
В качестве ответа запишите название этого фильма без указания года его выпуска.

In [ ]:
mask1 = rating_movies['year_release'] == 1999
rating_movies[mask].groupby('title')['rating'].mean().sort_values()


### Задание 8.3

Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [ ]:
mask2 = rating_movies['year_release'] == 2010
rating_movies[mask2].groupby('genres')['rating'].mean().sort_values()

### Задание 8.4

Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов? В качестве ответа запишите идентификатор этого пользователя.

In [ ]:
rating_movies.groupby('userId')['genres'].nunique().sort_values()

### Задание 8.5

Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
В качестве ответа укажите идентификатор этого пользователя.

In [ ]:
#сгрупируем пользоваелей по userId и найдем общее количество оценок каждого пользователя
num_rating = rating_movies.groupby('userId')['userId'].size()
#в полученном Series одно и ттоже имя  столбца и индекса. Присвои новое имя столбцу и заменим предыдущий Series.
num_rating.rename('Numeric',inplace=True)
#сгрупируем пользователей по userId и найдем среднюю оценку каждого пользователя
mean_rating = rating_movies.groupby('userId')['rating'].mean()
# Обьединим две таблицы по индексам
task_8 = pd.concat([num_rating,mean_rating], axis=1)
# Сортировка полученной таблицы в соотвествие с заданием.
task_8.sort_values(by=['Numeric','rating'], ascending=[True,False])

Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), можно воспользоваться методом agg() на сгруппированных данных.

In [ ]:
rating_movies.groupby('userId')['rating'].agg(['count','mean']).sort_values(by=['count','mean'],ascending=[True,False])

### Задание 8.6

Найдите сочетание жанров (genres) за 2018 год, которое имеет наибольший средний рейтинг (среднее по столбцу rating), и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [ ]:
# фильтр на 2018 год
mask_4 =rating_movies['year_release']==2018
# составим таблицу с количеством оценок и средним рейтингом для каждого сочетания из жанров
genres = rating_movies[mask_4].groupby('genres')['rating'].agg(['count','mean'])
# применим к полученой таблице фильтр на количество оценок больше 10 и сортируем столбец mean по убыванию
mask_5 = genres['count']>10
genres[mask_5].sort_values(by='mean',ascending=False)

### Задание 8.7

Добавьте в таблицу новый признак year_rating — год выставления оценки.

In [ ]:
# Приведем значения в столбце data к типу данных datatime
rating_movies['date'] = pd.to_datetime(rating_movies['date'], dayfirst=True)
#Добавим признак year_rating
rating_movies['year_rating'] = rating_movies['date'].dt.year

Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра.

In [ ]:
rating_movies.head()

In [ ]:
pivot_table = rating_movies.pivot_table(
    values='rating',
    index='genres',
    columns= 'year_rating',
    aggfunc='mean',
    fill_value=0
)


За весь период (с 1996 по 2018 год) сочетание жанров Action|Adventure ни разу не получало среднюю оценку ниже 3?

In [ ]:
pivot_table.loc['Action|Adventure'].sort_values()

Наилучшую оценку жанр Action|Adventure|Animation|Children|Comedy|IMAX получил в 2010 году.

In [28]:
pivot_table.loc['Action|Adventure|Animation|Children|Comedy|IMAX']

year_rating
1996    0.000000
1997    0.000000
1998    0.000000
1999    0.000000
2000    0.000000
2001    0.000000
2002    0.000000
2003    0.000000
2004    0.000000
2005    0.000000
2006    0.000000
2007    0.000000
2008    0.000000
2009    3.250000
2010    3.500000
2011    0.000000
2012    3.550000
2013    5.000000
2014    3.500000
2015    3.375000
2016    3.666667
2017    1.000000
2018    3.700000
Name: Action|Adventure|Animation|Children|Comedy|IMAX, dtype: float64

Среди сочетаний жанров, получивших наивысшую среднюю оценку в 2018 году, есть сочетание Animation|Children|Mystery.

In [66]:
# Способ №1: найти нивысшую среднею оценку в 2018 году и сравнить полученное значение с оценкой для жанра 
pivot_table.loc['Animation|Children|Mystery'][2018]==pivot_table[2018].max()
# Способ №2: собрать список жанров получивших наивысшую оценку в 2018 году. Проверить входит ли в этот список заданный жанр.
'Animation|Children|Mystery' in list(pivot_table[pivot_table[2018]==pivot_table[2018].max()][2018].index)

True

Для жанра Comedy прослеживается тенденция падения рейтинга с каждым годом (с 1996 по 2018).

In [67]:
pivot_table.loc['Comedy']

year_rating
1996    3.228571
1997    3.409091
1998    3.000000
1999    3.606061
2000    3.141291
2001    3.318408
2002    3.198556
2003    3.120066
2004    3.356877
2005    2.963325
2006    3.347534
2007    2.928187
2008    3.426667
2009    3.273292
2010    3.179825
2011    3.232877
2012    3.478477
2013    3.489474
2014    3.271429
2015    3.106183
2016    3.373431
2017    2.852668
2018    3.121296
Name: Comedy, dtype: float64

Прочитайте данные файлы, посмотрите на содержимое таблиц и проанализируйте его.

Подумайте, как связаны данные таблицы (какими ключевыми столбцами).

In [24]:
import pandas as pd
orders = pd.read_csv('data/task/orders.csv',sep=';')
products = pd.read_csv('data/task/products.csv',sep=';')
orders


,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3


In [25]:
products

,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR
5,86,"Носки Простые, муж",45,RUR
6,91,"Носки Честные, муж",50,RUR
7,103,"Носки Подарочные, муж",199,RUR
8,104,"Носки Подарочные, жен",249,RUR
9,124,Носки беговые Camino,999,RUR


Объедините заданные таблицы в таблицу orders_products, чтобы в результирующей таблице оказалась информация обо всех заказах, но не оказалось информации о продуктах, на которых заказов ещё не поступало.

Обьединние будем производить с помощью метода Merge:
- right — присоединяемая таблица. По умолчанию она является «правой»;
- how — параметр типа объединения. По умолчанию принимает значение 'inner';
- on — параметр, который определяет, по какому столбцу происходит объединение. Определяется автоматически, но рекомендуется указывать вручную;
- left_on — если названия столбцов в «левой» и «правой» таблицах не совпадают, то данный параметр отвечает за наименования ключевого столбца исходной таблицы;
- right_on — аналогично предыдущему, параметр отвечает за наименование ключевого столбца присоединяемой таблицы.

In [27]:
orders_product = orders.merge(
    products,
    how='left',
    left_on='ID товара',
    right_on='Product_ID'
)
orders_product

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86.0,"Носки Простые, муж",45.0,RUR
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104.0,"Носки Подарочные, жен",249.0,RUR
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104.0,"Носки Подарочные, жен",249.0,RUR
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104.0,"Носки Подарочные, жен",249.0,RUR
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,124.0,Носки беговые Camino,999.0,RUR
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,91.0,"Носки Честные, муж",50.0,RUR
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,103.0,"Носки Подарочные, муж",199.0,RUR
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,104.0,"Носки Подарочные, жен",249.0,RUR


### Задание 8.10

Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
В ответ запишите идентификатор этого покупателя (ID Покупателя).
Прибыль состоит только из оплаченных заказов и рассчитывается как количество купленного товара, умноженное на его цену.

In [33]:
# Добавим столбец Profit-суммарная прибыль
orders_product['profit']=orders_product['Количество']*orders_product['Price']
# Посмотрим какой покупатель принес наибольшую прибыль
orders_product[['ID Покупателя','profit']].sort_values(by='profit',ascending=False)

,ID Покупателя,profit
10,7,8999.0
14,5,7999.0
16,5,4999.0
11,7,4999.0
1,9,4500.0
13,7,2999.0
2,9,2490.0
3,8,1743.0
4,1,1245.0
6,5,999.0
